# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
files = "../output_data/Weather_cities.csv"
new_df = pd.read_csv(files)
new_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao joao da barra,-21.6403,-41.0511,75.34,84,15,6.64,BR,1649736243
1,1,thompson,55.7435,-97.8558,24.85,97,95,5.03,CA,1649736243
2,2,fairbanks,64.8378,-147.7164,39.54,54,28,2.59,US,1649736000
3,3,lujiang,31.2581,117.2817,84.00,43,74,12.73,CN,1649736244
4,4,ushuaia,-54.8000,-68.3000,35.24,89,46,6.85,AR,1649736171
...,...,...,...,...,...,...,...,...,...,...
559,559,paita,-5.0892,-81.1144,69.98,84,60,12.68,PE,1649736500
560,560,dickinson,29.4608,-95.0513,75.16,88,100,12.66,US,1649736208
561,561,westlock,54.1502,-113.8688,19.11,62,99,14.29,CA,1649736338
562,562,muswellbrook,-32.2667,150.9000,70.65,64,88,7.02,AU,1649736485


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [39]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map

location_pairs = new_df[["Lat", "Lng"]]
#check what type of obect humidity is
# new_df["Humidity"].dtype
#change it to a float
humidity = new_df["Humidity"].astype(float)
humidity

figure_1 = gmaps.figure()
heatmap = gmaps.heatmap_layer(location_pairs, weights=humidity, dissipating=False, max_intensity = 100, point_radius=2)

figure_1.add_layer(heatmap)
figure_1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
#Narrow down the Dataframe to find the ideal weather condition for a vacation

ideal_weather = new_df.loc[(new_df["Max Temp"] < 80) & (new_df["Max Temp"] > 70)]

ideal_weather = ideal_weather.loc[(ideal_weather["Wind Speed"] < 10)]

ideal_weather = ideal_weather.loc[(ideal_weather["Cloudiness"] == 0)]

ideal_weather = ideal_weather.dropna()

ideal_weather


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,84,shimoda,34.6667,138.9500,77.11,64,0,9.66,JP,1649736279
102,102,cabo san lucas,22.8909,-109.9124,71.62,70,0,5.01,MX,1649736199
116,116,busselton,-33.6500,115.3333,74.28,43,0,6.60,AU,1649736247
338,338,kerrville,30.0474,-99.1403,70.92,84,0,5.75,US,1649736291
459,459,khuzdar,27.7384,66.6434,79.45,19,0,5.59,PK,1649736454
507,507,najran,17.4924,44.1277,74.46,29,0,5.32,SA,1649736476
509,509,vredendal,-31.6683,18.5012,72.25,38,0,6.20,ZA,1649736477


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# Create dataframe and empty column
hotel_df = ideal_weather
Hotel_Name = []

#set parameters for the api search
params = {"radius": 5000,
          "types": "lodging",
         "key": g_key}

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [66]:
# Loop through dataframe and add nearby hotel result

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(url, params=params)
    
    hotel_info = response.json()
#     print(hotel_info)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
    except:
        print("Cities without a hotel", row["City"])
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
84,84,shimoda,34.6667,138.9500,77.11,64,0,9.66,JP,1649736279,Shimoda Tokyu Hotel
102,102,cabo san lucas,22.8909,-109.9124,71.62,70,0,5.01,MX,1649736199,Hotel Tesoro Los Cabos
116,116,busselton,-33.6500,115.3333,74.28,43,0,6.60,AU,1649736247,Observatory Guest House
338,338,kerrville,30.0474,-99.1403,70.92,84,0,5.75,US,1649736291,Inn Of The Hills Hotel & Conference Center
459,459,khuzdar,27.7384,66.6434,79.45,19,0,5.59,PK,1649736454,jhalwan Rent A Car Compny Moola Chotok
507,507,najran,17.4924,44.1277,74.46,29,0,5.32,SA,1649736476,محمد هادي ال فطيح
509,509,vredendal,-31.6683,18.5012,72.25,38,0,6.20,ZA,1649736477,Tharrakamma Guest House


In [67]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [69]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations)
figure_1.add_layer(marker_layer)

# Display figure
figure_1


Figure(layout=FigureLayout(height='420px'))